# Imports

In [ ]:
import os
import dotenv

dotenv.load_dotenv()

In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:


db = SQLDatabase.from_uri("sqlite:///Chinook.db") #being ideal for demos and testing ORM tools


llm = OpenAI(temperature=0, verbose=True)


db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
answer = db_chain.run("How many employees are there?")

In [ ]:
answer

# Case 1: Text-to-SQL query

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import create_sql_query_chain

In [ ]:
chain = create_sql_query_chain(OpenAI(temperature=0), db)
response = chain.invoke({"question":"How many employees are there"})
print(response)

In [ ]:
answer = db.run(response)

In [ ]:
eval(answer)[0][0]

# Case 2: Text-to-SQL query to Execution to Summarization

Some research papers have reported good performance when prompting with:

- A CREATE TABLE description for each table, which include column names, their types, etc
- Followed by three example rows in a SELECT statement

In [ ]:
# w

The query builder can be improved in several ways, such as:

- Customizing database description to your specific use case
- Hardcoding a few examples of questions and their corresponding SQL query in the prompt
- Using a vector database to include dynamic examples that are relevant to the specific user question

In [ ]:
# w

# Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer with a Summarization.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

Question: What is the total number of employees?
SQLQuery: SELECT COUNT(*) FROM Employee

Question: What is the country with the largest number of employees?
SQLQuery: SELECT Country, COUNT(*) AS EmployeeCount FROM Employee GROUP BY Country ORDER BY EmployeeCount DESC LIMIT 1;

Question: {input}"""




CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=TEMPLATE,
)

In [ ]:
# table_names = ['Album', 'Artist', 'Genre', 'MediaType', 'Track', 'Employee', 'Invoice', 'InvoiceLine', 'Playlist', 'PlaylistTrack', 'Customer']

In [ ]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db", include_tables=['Employee'], sample_rows_in_table_info=2)

llm = OpenAI(temperature=0, verbose=True)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=CUSTOM_PROMPT, use_query_checker=True, verbose=True)

In [ ]:
# print(db.table_info)

In [ ]:
db_chain.run("What are the available cities")

In [ ]:
db_chain.run("What are the distinct countries in the data")

In [ ]:
db_chain.run("Which city has the most employees")

In [ ]:
db_chain.run("select * from Employee")

In [ ]:
db_chain.run("show tables")